## PROYECTO 1 - MINERIA DE DATOS

In [2]:
# Importando librerias necesarias
import pandas as pd

In [21]:
# Cargando el data
data_2011 = pd.read_spss('matrimonio2011.sav')
data_2011 = pd.DataFrame(data_2011)

data_2012 = pd.read_spss('matrimonio2012.sav')
data_2012 = pd.DataFrame(data_2012)

data_2013 = pd.read_spss('matrimonio2013.sav')
data_2013 = pd.DataFrame(data_2013)

data_2014 = pd.read_spss('matrimonio2014.sav')
data_2014 = pd.DataFrame(data_2014)

data_2015 = pd.read_spss('matrimonio2015.sav')
data_2015 = pd.DataFrame(data_2015)

data_2016 = pd.read_spss('matrimonio2016.sav')
data_2016 = pd.DataFrame(data_2016)

data_2017 = pd.read_spss('matrimonio2017.sav')
data_2017 = pd.DataFrame(data_2017)

data_2018 = pd.read_spss('matrimonio2018.sav')
data_2018 = pd.DataFrame(data_2018)

data_2019 = pd.read_spss('matrimonio2019.sav')
data_2019 = pd.DataFrame(data_2019)

data_2020 = pd.read_spss('matrimonio2020.sav')
data_2020 = pd.DataFrame(data_2020)

data_2021 = pd.read_spss('matrimonio2021.sav')
data_2021 = pd.DataFrame(data_2021)


In [36]:
data_2011['Gethom'].value_counts()

Gethom
No indigena    35171
Indigena       27075
Ignorado       16040
Name: count, dtype: int64

In [37]:
# Limpieza data - 2011

data_2011['grupoEtnicoHombre'] = 0
GETHOM = list(set(gh for gh in data_2011['Gethom']))
for indx, gh in enumerate(GETHOM):
    if gh == 'No indigena':
        data_2011.loc[data_2011['Gethom'] == gh, "grupoEtnicoHombre"] = 1
    elif gh == 'Indigena':
        data_2011.loc[data_2011['Gethom'] == gh, "grupoEtnicoHombre"] = 2

data_2011['grupoEtnicoMujer'] = 0
GETMUJ = list(set(gh for gh in data_2011['Getmuj']))
for indx, gh in enumerate(GETMUJ):
    if gh == 'No indigena':
        data_2011.loc[data_2011['Getmuj'] == gh, "grupoEtnicoMujer"] = 1
    elif gh == 'Indigena':
        data_2011.loc[data_2011['Getmuj'] == gh, "grupoEtnicoMujer"] = 2



In [39]:
data_2012['GETMUJ'].value_counts()

GETMUJ
No indigena    39785
Indigena       28893
Ignorado       15575
Name: count, dtype: int64

In [50]:
# Limpieza data - 2012

data_2012['grupoEtnicoHombre'] = 0
GETHOM = list(set(gh for gh in data_2012['GETHOM']))
for indx, gh in enumerate(GETHOM):
    if gh == 'No indigena':
        data_2012.loc[data_2012['GETHOM'] == gh, "grupoEtnicoHombre"] = 1
    elif gh == 'Indigena':
        data_2012.loc[data_2012['GETHOM'] == gh, "grupoEtnicoHombre"] = 2


data_2012['grupoEtnicoMujer'] = 0
GETMUJ = list(set(gh for gh in data_2012['GETMUJ']))
for indx, gh in enumerate(GETMUJ):
    if gh == 'No indigena':
        data_2012.loc[data_2012['GETMUJ'] == gh, "grupoEtnicoMujer"] = 1
    elif gh == 'Indigena':
        data_2012.loc[data_2012['GETMUJ'] == gh, "grupoEtnicoMujer"] = 2

data_2012['grupoEtnicoMujer'].value_counts()

grupoEtnicoMujer
1    39785
2    28893
0    15575
Name: count, dtype: int64

In [42]:
data_2013['PUEMUJ'].value_counts()

PUEMUJ
Mestizo / Ladino    42516
Maya                27679
Ignorado            10170
Otro                  342
Garífuna               26
Xinka                  17
Name: count, dtype: int64

In [44]:
# Limpieza data - 2013

data_2013['grupoEtnicoHombre'] = 0
GETHOM = list(set(gh for gh in data_2013['PUEHOM']))
for indx, gh in enumerate(GETHOM):
    if gh == 'Mestizo / Ladino':
        data_2013.loc[data_2013['PUEHOM'] == gh, "grupoEtnicoHombre"] = 1
    elif gh == 'Maya':
        data_2013.loc[data_2013['PUEHOM'] == gh, "grupoEtnicoHombre"] = 2
    elif gh == 'Garífuna':
        data_2013.loc[data_2013['PUEHOM'] == gh, "grupoEtnicoHombre"] = 2
    elif gh == 'Xinca':
        data_2013.loc[data_2013['PUEHOM'] == gh, "grupoEtnicoHombre"] = 2


data_2013['grupoEtnicoMujer'] = 0
GETMUJ = list(set(gh for gh in data_2013['PUEMUJ']))
for indx, gh in enumerate(GETMUJ):
    if gh == 'Mestizo / Ladino':
        data_2013.loc[data_2013['PUEMUJ'] == gh, "grupoEtnicoMujer"] = 1
    elif gh == 'Maya':
        data_2013.loc[data_2013['PUEMUJ'] == gh, "grupoEtnicoMujer"] = 2
    elif gh == 'Garífuna':
        data_2013.loc[data_2013['PUEMUJ'] == gh, "grupoEtnicoMujer"] = 2
    elif gh == 'Xinca':
        data_2013.loc[data_2013['PUEMUJ'] == gh, "grupoEtnicoMujer"] = 2

data_2013['grupoEtnicoHombre'].value_counts()

        

grupoEtnicoHombre
1    44865
2    29428
0     6457
Name: count, dtype: int64

In [11]:
data_2014['PUEHOM'].value_counts()

PUEHOM
Mestizo / Ladino    45464
Maya                29188
Ignorado             4385
Otro                  408
Garífuna               30
Xinka                  21
Name: count, dtype: int64

In [48]:
# Limpieza data - 2014

data_2014['grupoEtnicoHombre'] = 0
GETHOM = list(set(gh for gh in data_2014['PUEHOM']))
for indx, gh in enumerate(GETHOM):
    if gh == 'Mestizo / Ladino':
        data_2014.loc[data_2014['PUEHOM'] == gh, "grupoEtnicoHombre"] = 1
    elif gh == 'Maya':
        data_2014.loc[data_2014['PUEHOM'] == gh, "grupoEtnicoHombre"] = 2
    elif gh == 'Garífuna':
        data_2014.loc[data_2014['PUEHOM'] == gh, "grupoEtnicoHombre"] = 2
    elif gh == 'Xinca':
        data_2014.loc[data_2014['PUEHOM'] == gh, "grupoEtnicoHombre"] = 2

data_2014['grupoEtnicoMujer'] = 0
GETMUJ = list(set(gh for gh in data_2014['PUEMUJ']))
for indx, gh in enumerate(GETMUJ):
    if gh == 'Mestizo / Ladino':
        data_2014.loc[data_2014['PUEMUJ'] == gh, "grupoEtnicoMujer"] = 1
    elif gh == 'Maya':
        data_2014.loc[data_2014['PUEMUJ'] == gh, "grupoEtnicoMujer"] = 2
    elif gh == 'Garífuna':
        data_2014.loc[data_2014['PUEMUJ'] == gh, "grupoEtnicoMujer"] = 2
    elif gh == 'Xinca':
        data_2014.loc[data_2014['PUEMUJ'] == gh, "grupoEtnicoMujer"] = 2



grupoEtnicoHombre
1    45464
2    29218
0     4814
Name: count, dtype: int64

In [12]:
data_2015['PUEHOM'].value_counts()

PUEHOM
Mestizo / Ladino    44756
Maya                29146
Ignorado             4322
Otro                  911
Garífuna               22
Xinka                  20
Name: count, dtype: int64

In [13]:
data_2016['PUEHOM'].value_counts()

PUEHOM
Mestizo / Ladino    40262
Maya                25028
Ignorado             3333
Otro                  938
Garífuna               31
Xinka                  21
Name: count, dtype: int64

In [14]:
data_2017['PUEHOM'].value_counts()

PUEHOM
Ladino / Mestizo    40366
Maya                26024
Ignorado             3952
Otro                 1215
Garífuna               20
Xinca                  20
Name: count, dtype: int64

In [15]:
data_2018['PUEHOM'].value_counts()

PUEHOM
Ladino / Mestizo    41016
Maya                27531
Ignorado             4448
Otro                 1723
Xinca                  45
Garífuna               14
Name: count, dtype: int64

In [16]:
data_2019['PUEHOM'].value_counts()

PUEHOM
Ladino / Mestizo    41811
Maya                28147
Ignorado             4934
Otro                 1963
Xinca                  57
Garífuna               16
Name: count, dtype: int64

In [17]:
data_2020['PUEHOM'].value_counts()

PUEHOM
Ladino / Mestizo    29945
Maya                21378
Ignorado             4440
Otro                 1574
Xinka                  38
Garífuna               12
Name: count, dtype: int64

In [18]:
data_2021['PUEHOM'].value_counts()

PUEHOM
Ladino / Mestizo    45370
Maya                32652
Ignorado             6795
Otro                 2507
Xinka                 126
Garífuna               30
Name: count, dtype: int64